# Init the AutoML program

In [1]:
import numpy as np
import os
import tensorflow as tf

def train(train_dir, validation_dir=None, batch_size=32, initial_epachs=1, finetune_epochs=1, image_size=(160,160)):
    train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                            shuffle=True,
                                                            batch_size=batch_size,
                                                            image_size=image_size)
    print(train_dataset)

    if validation_dir:
      validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,
                                                                  shuffle=True,
                                                                  batch_size=batch_size,
                                                                  image_size=image_size)
      batches = tf.data.experimental.cardinality(validation_dataset)
      if batches == 1:
        test_dataset = validation_dataset
      elif batches < 5:
        test_dataset = validation_dataset.take(batches // 2)
        validation_dataset = validation_dataset.skip(batches // 2)
      else:
        test_dataset = validation_dataset.take(batches // 5)
        validation_dataset = validation_dataset.skip(batches // 5)
    else:
      test_dataset = None
      validation_dataset = None

    print('Number of trian batches: %d' % tf.data.experimental.cardinality(train_dataset))
    if validation_dataset:
      print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
      print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))


    AUTOTUNE = tf.data.AUTOTUNE
    train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
    if validation_dataset:
      validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
      test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

    data_augmentation = tf.keras.Sequential([
      tf.keras.layers.RandomFlip('horizontal'),
      tf.keras.layers.RandomRotation(0.2),
    ])

    preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
    # rescale = tf.keras.layers.Rescaling(1./127.5, offset=-1)

    # Create the base model from the pre-trained model MobileNet V2
    image_shape = image_size + (3,)
    base_model = tf.keras.applications.MobileNetV2(input_shape=image_shape,
                                               include_top=False,
                                               weights='imagenet')

    image_batch, label_batch = next(iter(train_dataset))
    feature_batch = base_model(image_batch)
    # print(feature_batch.shape)


    # Feature Extaction
    base_model.trainable = False
    # Let's take a look at the base model architecture
    # base_model.summary()
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
    feature_batch_average = global_average_layer(feature_batch)
    # print(feature_batch_average.shape)
    prediction_layer = tf.keras.layers.Dense(1)
    prediction_batch = prediction_layer(feature_batch_average)
    # print(prediction_batch.shape)
    inputs = tf.keras.Input(shape=image_shape)
    x = data_augmentation(inputs)
    x = preprocess_input(x)
    x = base_model(x, training=False)
    x = global_average_layer(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs, outputs)
    base_learning_rate = 0.0001
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
    # model.summary()
    # len(model.trainable_variables)

    # if validation_dataset:
    #   loss0, accuracy0 = model.evaluate(validation_dataset)
    #   print("initial loss: {:.2f}".format(loss0))
    #   print("initial accuracy: {:.2f}".format(accuracy0))

    history = model.fit(train_dataset,
                    epochs=initial_epachs,
                    validation_data=validation_dataset)
    # acc = history.history['accuracy']
    # loss = history.history['loss']
    # if validation_dataset:
    #   val_acc = history.history['val_accuracy']
    #   val_loss = history.history['val_loss']

    # Finetune
    base_model.trainable = True
    # print("Number of layers in the base model: ", len(base_model.layers))
    fine_tune_at = 100
    for layer in base_model.layers[:fine_tune_at]:
      layer.trainable =  False
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate/10),
              metrics=['accuracy'])
    # model.summary()
    # len(model.trainable_variables)

    total_epochs =  initial_epachs + finetune_epochs

    history_fine = model.fit(train_dataset,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=validation_dataset)
    # acc += history_fine.history['accuracy']
    # loss += history_fine.history['loss']
    # if validation_dataset:
    #   val_acc += history_fine.history['val_accuracy']
    #   val_loss += history_fine.history['val_loss']

    if test_dataset:
      loss, accuracy = model.evaluate(test_dataset)
      print('Test accuracy :', accuracy)

      # Retrieve a batch of images from the test set
      image_batch, label_batch = test_dataset.as_numpy_iterator().next()
      predictions = model.predict_on_batch(image_batch).flatten()

      # Apply a sigmoid since our model returns logits
      predictions = tf.nn.sigmoid(predictions)
      predictions = tf.where(predictions < 0.5, 0, 1)

      print('Predictions:\n', predictions.numpy())
      print('Labels:\n', label_batch)

# Train my Data

## Upload my Data

In [ ]:
from google.colab import files
import os

uploaded = files.upload()

filename=None
for fn in uploaded.keys():
  filename = fn  

!unzip $filename
base, ext = os.path.splitext(filename)
train_dir = f'{base}/train'
validation_dir = f'{base}/validation'

print(f"train dir: {train_dir}")
print(f"validation dir: {validation_dir}")

## Train

In [ ]:
BATCH_SIZE = 32
INITIAL_EPOCHS = 5
FINETUNE_EPOCHS = 20
IMG_SIZE = (160, 160)
train(train_dir, validation_dir, initial_epachs=INITIAL_EPOCHS, finetune_epochs=FINETUNE_EPOCHS)